In [1]:
import pandas as pd
import matplotlib.pyplot as plt

/tmp/ipykernel_84091/1492400551.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def correct_string(line: str, A_before: int) -> (list, int):
    """
    Turns one line of the string into a list with values for all these cols (15)
    'N', 'Z', 'A', 'El', 'Orig', 'Mass Excess (keV)', 'ME Unc',
       'Binding Energy/A (keV)', 'BE/A Unc', 'Beta-decay Type',
       'Beta-decay Energy (keV)', 'BE Unc', 'N Protons',
       'Atomic Mass (micro-u)', 'AM Unc'
    """
    N = int(line[2:9].strip())
    Z = int(line[10:14].strip())
    
    A_test = line[15:19].strip()
    if A_test == "":  # Can be empty
        A = A_before
    else:
        A = int(A_test)
    
    El = line[20:22].strip()
    Orig = line[23:27].strip()
    mass_excess = line[28:41].strip()  # missing convertion for # is str for now
    mass_excess_unc = line[42:52].strip() # missing convertion for # is str for now
    b_energya = line[53:63].strip() # missing convertion for # is str for now
    b_energya_unc = line[63:72].strip() # missing convertion for # is str for now
    b_type = line[72:75].strip()
    b_energy = line[75:86].strip() # missing convertion for # and * is str for now
    b_energy_unc = line[87:95].strip() # missing convertion for # and * is str for now
    n_protons = int(line[96:99].strip())
    a_mass = line[99:112].strip() # missing convertion for # and * is str for now
    am_unc = line[112:].strip() # missing convertion for # and * is str for now
    
    output_values = [
        N, Z, A, El, Orig, mass_excess, mass_excess_unc, b_energya, b_energya_unc,
        b_type, b_energy, b_energy_unc, n_protons, a_mass, am_unc
    ]
    return output_values, A
    

In [3]:
file1 = open('massround.mas20.txt', 'r')
Lines = file1.readlines()

In [4]:
pd_cols = Lines[32].rsplit()[1:]
df = pd.DataFrame(columns=pd_cols)
df = df.rename(columns={"Mass": "Mass Excess (keV)", "Excess": "ME Unc", "Binding": "Binding Energy/A (keV)",
                  "Energy/A": "BE/A Unc", "Beta-decay": "Beta-decay Type",
                   "Energy": "Beta-decay Energy (keV)", "Atomic": "BE Unc"})
df = df.iloc[:,:-1]
df["N Protons"] = None
df["Atomic Mass (micro-u)"] = None
df["AM Unc"] = None

In [5]:
line_number = 34
a_before = 0

for i in range(len(Lines[34:])):
    try:
        value_list, a_temp = correct_string(Lines[line_number + i], a_before)
        a_before = a_temp  # This values sometimes does not change so it is saved between loops
        df.loc[i] = value_list
    except Exception as e:
        print(i, e)

In [6]:
print("Now you have the file massround.mas20.txt in the pd.DataFrame() -> df")

Now you have the file massround.mas20.txt in the pd.DataFrame() -> df


In [11]:
df.loc[(df["Z"] == 82) & (df["A"] == 208)]

,N,Z,A,El,Orig,Mass Excess (keV),ME Unc,Binding Energy/A (keV),BE/A Unc,Beta-decay Type,Beta-decay Energy (keV),BE Unc,N Protons,Atomic Mass (micro-u),AM Unc
2841,126,82,208,Pb,,-21748.5,1.1,7867.453,0.006,,*,,207,976652.0,1.2
